In [3]:
import pandas as pd
import numpy as np

In [4]:
df2 = pd.read_csv("sample_submission.csv")
df2

,query,video_id
0,query_0,video_0
1,query_0,video_1
2,query_0,video_2
3,query_0,video_3
4,query_0,video_4
...,...,...
9995,query_1999,video_0
9996,query_1999,video_1
9997,query_1999,video_2
9998,query_1999,video_3


In [5]:
df1 = pd.read_csv("submission_queries.csv")
df1

,query
0,экстрасенсы битва сильнейших 2 сезон
1,битва экстрасенсов 2
2,Экстрасенсы
3,битва сильнейших 19 сезон
4,битва экстрасенсов 20 сезон
...,...
1995,ПОЛИКЛИНИКА в этом ведре и есть счастье ?
1996,Как избавиться от страхов и начать бизнес ?
1997,Как и где продавать аромасвечи ?
1998,Ответы на Ваши комментарии! Аромасвечи


In [15]:
# Оставим только слова
df = df1['query']

rus_alph = set(list('йцукенгшщзхъфывапролджэёячсмитьбю0123456789'))
drop_not_letters = lambda s: ''.join(list(filter(lambda c: c in rus_alph or c == ' ', s)))

df = [drop_not_letters(comm.lower()) for comm in df]

# уберем пустые комменты
df = list(filter(lambda comm: len(comm) > 0, df))
print(len(df))

df[0:10]

1967


['экстрасенсы битва сильнейших 2 сезон',
 'битва экстрасенсов 2',
 'экстрасенсы',
 'битва сильнейших 19 сезон',
 'битва экстрасенсов 20 сезон',
 'битва экстрасенсов',
 'битва сильнейших 19 сезон',
 'экстрасенсы битва сильнейших',
 'звезды в дубае',
 'выжить в дубае звездам прислали верблюжий навоз']

In [41]:
"""def preprocess_text(text: str) -> str:
    text = text.lower()
    is_allowed_char = lambda c: c.isalpha() or c == ' '
    text = ''.join(list(filter(is_allowed_char, text)))
    return text

texts = [preprocess_text(text) for text in df]

texts[:10]"""

['экстрасенсы битва сильнейших  сезон',
 'битва экстрасенсов ',
 'экстрасенсы',
 'битва сильнейших  сезон',
 'битва экстрасенсов  сезон',
 'битва экстрасенсов',
 'битва сильнейших  сезон',
 'экстрасенсы битва сильнейших',
 'звезды в дубае',
 'выжить в дубае звездам прислали верблюжий навоз']

In [16]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

ru_stopwords = stopwords.words('russian')
print(len(ru_stopwords))

151


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
ru_stopwords = set(ru_stopwords)

def preprocess_text(text: str) -> str:
    '''text = text.lower()
    is_allowed_char = lambda c: c.isalpha() or c == ' '
    text = ''.join(list(filter(is_allowed_char, text)))'''

    is_stopword = lambda word: word not in ru_stopwords
    text = ' '.join(list(filter(is_stopword, text.split())))
    return text


texts = [preprocess_text(text) for text in df]

texts[:10]

['экстрасенсы битва сильнейших 2 сезон',
 'битва экстрасенсов 2',
 'экстрасенсы',
 'битва сильнейших 19 сезон',
 'битва экстрасенсов 20 сезон',
 'битва экстрасенсов',
 'битва сильнейших 19 сезон',
 'экстрасенсы битва сильнейших',
 'звезды дубае',
 'выжить дубае звездам прислали верблюжий навоз']

In [10]:
from w_dict_manual import words_dictionary_manual

In [17]:
answer  = [text.split(' ') for text in df]
print(answer)

[['экстрасенсы', 'битва', 'сильнейших', '2', 'сезон'], ['битва', 'экстрасенсов', '2'], ['экстрасенсы'], ['битва', 'сильнейших', '19', 'сезон'], ['битва', 'экстрасенсов', '20', 'сезон'], ['битва', 'экстрасенсов'], ['битва', 'сильнейших', '19', 'сезон'], ['экстрасенсы', 'битва', 'сильнейших'], ['звезды', 'в', 'дубае'], ['выжить', 'в', 'дубае', 'звездам', 'прислали', 'верблюжий', 'навоз'], ['дубай'], ['выжить', 'в', 'дубае', '2'], ['выжить', 'в'], ['выжить', 'в', 'дубае', '2', 'выпуск'], ['выжить', 'в', 'дубае'], ['выжить', 'в', 'дубае', '2', 'выпуск'], ['тнт'], ['выжить', 'в', 'дубае', '2', 'выпуск'], ['звёзды', 'в', 'дубае'], ['выжить', 'в', 'дубае', '2', 'серия', ''], ['дубае'], ['в', 'дубае'], ['звезды', 'в', 'дубаи'], ['выжить', 'в', 'дуб'], ['звёзды', 'в', 'дубае'], ['', 'выжить', 'в', 'дубае'], ['жить', 'в', 'дубае'], ['выжить', 'в', 'дубе'], ['шоу', 'воли'], ['большое', 'шоу', '3', 'сезон'], ['выжить', 'в', 'дубаях'], ['выжить', 'в', 'дубае', ''], ['', 'в', 'дубае'], ['в', 'дубаи'

In [ ]:
y_pred = []
for lst in answer:
  pred = []
  for el in lst:
    if el in words_dictionary_manual:
      for zn in words_dictionary_manual[el]:
        pred.append(zn)
    y_pred.append(pred[-5:])

y_pred

In [32]:
def metric(lst1, lst2):
    kol_sovpadenie = 0
    for i, j in zip(lst1, lst2):
        kol_sovpadenie += len(set(i) & set(j))
    return kol_sovpadenie/(5*len(lst1))


lst1 = [['v1', 'v2', 'v3', 'v4', 'v5'], ['v6', 'v7', 'v8', 'v9', 'v10'], ['v11', 'v12', 'v13', 'v14', 'v15']]
lst2 = [['v2', 'v4', 'v6', 'v8', 'v10'], ['v1', 'v3', 'v5', 'v7', 'v9'], ['v11', 'v12', 'v13', 'v14', 'v15']]
print(metric(lst1, lst2))

0.6


In [33]:
metric(y_pred, df2['video_id'])

0.0